In [1]:
import datetime as dt
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [2]:
date = pd.date_range(dt.date(2019,1,1).strftime("%Y-%m-%dT%H:%M:%S+00:00"), periods=365, freq='D', name="date")
dataframe = pd.DataFrame(date, columns = ["date", "value"])
dataframe.fillna(0, inplace=True)

In [3]:
html = urlopen(
    "https://en.wikipedia.org/wiki/2019_in_spaceflight#Orbital_launches"
).read().decode('utf-8')

soup = BeautifulSoup(html, features='lxml')
orbitTable = soup.find("table", class_="wikitable")

good_answers = ['Successful', 'Operational', 'En Route']
# the two style that answers ('Successful', 'Operational', 'En Route') resides at
style_options = ["background: #e8eeee", "background: #f0f6f6; font-size: small;"]

# find all <tr> tags
trs = orbitTable.find_all("tr")

i = 0
while i < len(trs):
    tr = trs[i]
    date_td = tr.contents[1]
    
    # where the date locates
    if date_td.name == "td" and not tr.get("style", None) and date_td.get("rowspan", None):
        key = date_td.contents[0].contents[0].rstrip()
        key_datetime = dt.datetime.strptime(key, "%d %B").timetuple().tm_yday # time to index
        
        i += 1
        flag = False
        # find all answers
        while i < len(trs) and trs[i].get("style", None) in style_options:
            if trs[i].contents[-1].contents[0].rstrip() in good_answers: # if it has good answer
                flag = True
            i += 1
        
        # consider as good launch
        if flag:
            dataframe.loc[key_datetime-1, "value"] += 1
    else:        
        i += 1

In [4]:
# to csv file
dataframe.to_csv("output.csv", index=False)